# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Read the cities_df.csv file into a DataFrame
cities_df = pd.read_csv("output_data/cities_df.csv")
cities_df = cities_df.drop(columns=["Unnamed: 0"], axis=1)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,petropavlovsk-kamchatskiy,53.0452,158.6483,13.33,67.0,75.0,2.00,RU,1.663051e+09
1,saskylakh,71.9167,114.0833,1.93,97.0,100.0,1.71,RU,1.663051e+09
2,ushuaia,-54.8000,-68.3000,5.81,52.0,20.0,5.66,AR,1.663051e+09
3,port alfred,-33.5906,26.8910,15.10,72.0,15.0,2.24,ZA,1.663051e+09
4,hithadhoo,-0.6000,73.0833,27.82,70.0,12.0,2.64,MV,1.663051e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["Lat", "Lng"]].astype(float)
# Convert Humidity Rate to float and store
# HINT: be sure to handle NaN values
humidity_rate = cities_df["Humidity"].astype(float)


In [8]:
# Create a Humidity Heatmap layer
# Customise the size of the figure
figure_layout = {
    'width': '640px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(30.0, 20.0), zoom_level=1.2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate,
                                    dissipating=False, max_intensity=350,
                                    point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the DataFrame to find your ideal weather condition.
# For example:
# A max temperature lower than 27 degrees but higher than 21.
# Wind speed less than 4.5 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
ideal_weather_df = cities_df.loc[(cities_df["Max Temp"] < 27) & (cities_df["Max Temp"] > 23) & (cities_df["Wind Speed"] < 4.5) & (cities_df["Cloudiness"] == 0) & (cities_df["Humidity"] < 60)]
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.head()
print(len(ideal_weather_df))    

9


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#Add Hotel Name column to the DataFrame and store it in a new DataFrame
ideal_weather_df["Hotel Name"] = ""
hotel_df = ideal_weather_df


In [16]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
29,tabriz,38.0800,46.2919,25.70,29.0,0.0,0.00,IR,1.663051e+09,
121,pingzhuang,42.0372,119.2889,23.53,47.0,0.0,3.90,CN,1.663051e+09,
123,binga,-17.6203,27.3414,26.09,26.0,0.0,3.79,ZW,1.663051e+09,
267,vangaindrano,-23.3500,47.6000,24.05,56.0,0.0,4.14,MG,1.663051e+09,
288,bundaberg,-24.8500,152.3500,25.12,43.0,0.0,1.34,AU,1.663051e+09,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Customise the size of the figure
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(30.0, 31.0), zoom_level=1.5)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)  #, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…